In [1]:
import metapack as mp
doc = mp.jupyter.open_package()

r = doc.reference('census_planning_database') 

tracts =  doc.reference('tract_boundaries').geoframe()

comm = doc.reference('communities').geoframe()


In [3]:
communities_pkg = mp.open_package('http://library.metatab.org/sandiegodata.org-communities-2018-5.zip')
communities_pkg


In [4]:
communities_pkg.resource('tract_links')

Header,Type,Description
geoid,string,"Tract geoid, ACS format"
city_name,string,"City name, or “County’"
city_code,string,"City code, or “CN” for the county"
community_name,string,City or county community
community_cpcode,integer,Community plan code
community_code,string,Text code
geometry,string,Tract boundary geometry


In [8]:
communities = communities_pkg.resource('tract_links').dataframe().drop('geometry',axis=1).fillna('')

In [9]:
communities.head()

,geoid,city_name,city_code,community_name,community_cpcode,community_code
0,14000US06073020309,County,CN,North County Metro,190900,NOR
1,14000US06073020207,Escondido,ES,,0,
2,14000US06073020211,Escondido,ES,,0,
3,14000US06073020209,Escondido,ES,,0,
4,14000US06073020206,Escondido,ES,,0,


In [10]:
doc

In [11]:
doc.resource('sandiego_planning_db')

Header,Type,Description
geoid,string,
flag,string,
land_area,number,
aian_land,integer,
urbanized_area_pop_cen_2010,integer,
urban_cluster_pop_cen_2010,integer,
rural_pop_cen_2010,integer,
tot_population_cen_2010,integer,
tot_population_acs_12_16,integer,
tot_population_acsmoe_12_16,integer,


In [12]:
pdb  = doc.resource('sandiego_planning_db').dataframe()

In [15]:
pdb.set_index('geoid').join(communities.set_index('geoid')).head().T

geoid,14000US06073000100,14000US06073000201,14000US06073000202,14000US06073000300,14000US06073000400
flag,NaN,NaN,NaN,NaN,NaN
land_area,0.595,0.334,0.506,0.349,0.456
aian_land,0,0,0,0,0
urbanized_area_pop_cen_2010,3029,1801,4208,4732,3669
urban_cluster_pop_cen_2010,0,0,0,0,0
rural_pop_cen_2010,0,0,0,0,0
tot_population_cen_2010,3029,1801,4208,4732,3669
tot_population_acs_12_16,2773,2158,4828,4946,3916
tot_population_acsmoe_12_16,185,241,415,405,306
males_cen_2010,1518,845,2131,2460,2017
